In [1]:
import re
import json
import pandas as pd
from typing import Optional, Dict, List, Tuple
from IPython.display import display
from datetime import datetime
import time
from dotenv import load_dotenv
import os

load_dotenv("./api_key.env")
GITHUB_TOKEN = os.getenv("GITHUB_API_KEY")

# Import the Hao-Li AIDev datasets

In [2]:
# Repositories
repo_df = pd.read_parquet("hf://datasets/hao-li/AIDev/repository.parquet")

# Pull Request
pr_df = pd.read_parquet("hf://datasets/hao-li/AIDev/pull_request.parquet")

# 1. Prepare the Dataset

In [3]:
# Filter the repository data for 'Java' language
java_repo_df = repo_df[repo_df['language'] == 'Java'].copy()
java_repo_select_df = java_repo_df[['id', 'full_name']]

# Join Repo and PR table based on repo id
merged_pr_df = pr_df.merge(
    java_repo_select_df,
    left_on='repo_id',
    right_on='id',
    how='inner'
)

# clean up extra attribute
merged_pr_df = merged_pr_df.drop(columns=['id_y'])
merged_pr_df = merged_pr_df.rename(columns={'id_x':'id'})

# Filter PRs that were rejected (not merged) and create a new attribute
accepted_prs = merged_pr_df[merged_pr_df['merged_at'].notnull()]
rejected_prs = merged_pr_df[merged_pr_df['merged_at'].isnull()]

# Prepare for Merge: Rename the key column
accepted_prs = accepted_prs[['full_name', 'number','user']]
rejected_prs = rejected_prs[['full_name', 'number','user']]

# print to csv for checking
accepted_prs.to_csv("accepted_PR_dev.csv", index=False)
rejected_prs.to_csv("rejected_PR_dev.csv", index=False)

## 1.1. Split the full_name of repo into owner and repo name

In [4]:
# ============================================================
# Helper: Split the name and put it in a List of Dict (not needed but ehh accidentally made the method like that)
# ============================================================
def process_repositories(pr_df):
    """
    Filters the DataFrame by status, splits the full_name, and creates a 
    list of (owner, repo) tuples for processing.
    """
    
    # 1. Split the 'full_name' column into 'owner' and 'repo' columns
    split_df = pr_df['full_name'].str.split('/', n=1, expand=True)
    split_df.columns = ['owner', 'repo']
    
    # 2. Combine the split columns and the 'number' column into a list of tuples
    combined_df = pd.concat([split_df, pr_df['number'], pr_df['user']], axis=1) # use axis=1 to apply the tuple creation row-wise across the three columns
    unique_df = combined_df.drop_duplicates(subset=['owner', 'repo', 'user'])   # Drop duplicates based on the three columns
    
    # 
    repositories = unique_df.apply(tuple, axis=1).tolist()
    print(repositories[:5])
    
    return repositories


ACCEPTED_PULL_REQUEST = process_repositories(accepted_prs)
REJECTED_PULL_REQUEST = process_repositories(rejected_prs)

[('dotCMS', 'core', 32609, 'spbolton'), ('apache', 'pulsar', 24542, 'Apurva007'), ('microsoft', 'ApplicationInsights-Java', 4293, 'Copilot'), ('microsoft', 'typespec', 7783, 'Copilot'), ('valkey-io', 'valkey-glide', 4262, 'Copilot')]
[('dotCMS', 'core', 32656, 'spbolton'), ('hyperledger', 'besu', 8904, 'jflo'), ('MeteorDevelopment', 'meteor-client', 5568, 'claudecodebyia'), ('apache', 'pulsar', 24145, 'alexander-nailed-it'), ('microsoft', 'typespec', 7651, 'Copilot')]


# 2. Helper code block to limit the API rate request

In [ ]:
import time
import requests
import requests_cache

def safe_request(method, url, headers=None, params=None, timeout=10, sleep_between=0.4):
    """
    A rate-limit-safe GitHub request wrapper that handles:
    - Primary rate limits (5000/hour)
    - Secondary abuse limits (burst protection)
    - GET and HEAD requests
    """
    while True:
        response = requests.request(method, url, headers=headers, params=params, timeout=timeout)

        # ============================================================
        # Rate Limit handling
        # ============================================================
        # Primary rate limit
        remaining = int(response.headers.get("X-RateLimit-Remaining", 1))
        reset_ts = int(response.headers.get("X-RateLimit-Reset", time.time()))

        if remaining == 0:
            wait = max(reset_ts - int(time.time()), 10)
            print(f"[Primary Limit] Waiting {wait} seconds...")
            time.sleep(wait)
            continue

        # Secondary rate limit (abuse detection)
        if response.status_code == 403:
            print("[Secondary Limit] Hit GitHub abuse limit. Backing off 60 seconds...")
            time.sleep(60)
            continue
        
        # ============================================================
        # API cache
        # ============================================================
        # Check if the response came from the cache
        if hasattr(response, 'from_cache') and response.from_cache:
            print(f"[CACHE] Hit for {url}")
            # Skip the time.sleep(sleep_between) if it came from the cache
            return response

        # ============================================================
        # Network handling
        # ============================================================
        # Success or other errors handled normally
        if not response.ok:
            response.raise_for_status()

        # Small delay prevents triggering secondary limit
        time.sleep(sleep_between)

        return response


# 3. Git API to extract metrics 

In [ ]:
# ============================================================
# Helper 0: Total Commit Counts for that PR
# ============================================================
def get_commit_count(owner, repo, headers):
    """Retrieves the total number of commits for the entire repository."""
    url = f"https://api.github.com/repos/{owner}/{repo}/commits"
    params = {"per_page": 1}
    
    try:
        # Use HEAD request and pagination trick for total count
        response = safe_request("HEAD", url, headers=headers, params=params)
        response.raise_for_status()
        link_header = response.headers.get('Link')

        if link_header:
            last_page_match = re.search(r'page=(\d+)>; rel="last"', link_header)
            if last_page_match:
                return int(last_page_match.group(1))
        
        # Fallback for very small repos
        return 0 
        
    except requests.exceptions.RequestException:
        return 0
    
# ============================================================
# Helper 1: Developer PR Count for that Repo
# ============================================================
def get_developer_pr_count(owner: str, repo: str, developer_login: str, headers: Dict) -> int:
    """
    Retrieves the total number of PRs submitted by the developer in the repo 
    using the GitHub Search API (which correctly handles author filtering).
    """
    # Use the Search Issues endpoint, which allows filtering by author and type:pr
    url = "https://api.github.com/search/issues"
    
    # Construct the query: repo:owner/repo is:pr author:developer_login
    query = f"repo:{owner}/{repo} is:pr author:{developer_login}"
    
    # We only need the total_count, so we request a minimal response (per_page=1)
    params = {"q": query, "per_page": 1}
    
    response = safe_request("GET", url, headers=headers, params=params)
    if response and response.status_code == 200:
        try:
            data = response.json()
            # The search API returns the total count directly in the response body
            return data.get('total_count', 0)
        except json.JSONDecodeError:
            print("   [Error] Could not decode JSON response from Search API.")
            return 0
    
    return 0

# ============================================================
# Helper 2: Followers and Account Age (Active Duration)
# ============================================================
def get_developer_social_metrics(developer_login: str, headers: Dict) -> Dict:
    """
    Fetches the number of followers and the account creation date.
    """
    url = f"https://api.github.com/users/{developer_login}"
    
    response = safe_request("GET", url, headers=headers)
    if response and response.status_code == 200:
        user_data = response.json()
        
        # Calculate Active Duration
        created_at_str = user_data.get('created_at')
        account_age_days = 0
        if created_at_str:
            created_date = datetime.strptime(created_at_str, '%Y-%m-%dT%H:%M:%SZ')
            account_age_days = (datetime.now() - created_date).days
            
        return {
            "Followers": user_data.get('followers', 0),
            "Account_Created_At": created_at_str,
            "Account_Active_Duration_Days": account_age_days
        }
    return {
        "Followers": 0,
        "Account_Created_At": None,
        "Account_Active_Duration_Days": 0
    }

# ============================================================
# Helper 3: Global Developer PR Count (Across all Repos)
# ============================================================
def get_developer_global_pr_count(developer_login: str, headers: Dict) -> int:
    """
    Retrieves the total number of PRs submitted by the developer globally 
    using the GitHub Search API without the repository filter.
    """
    url = "https://api.github.com/search/issues"
    
    # Construct the query: is:pr author:developer_login (No repo filter)
    query = f"is:pr author:{developer_login}"
    
    params = {"q": query, "per_page": 1}
    
    response = safe_request("GET", url, headers=headers, params=params)
    if response and response.status_code == 200:
        try:
            data = response.json()
            return data.get('total_count', 0)
        except json.JSONDecodeError:
            print("   [Error] Could not decode JSON response from Global Search API.")
            return 0
    
    return 0

# ============================================================
# Helper Total: All the Developer Characteristics 
# ============================================================
def get_developer_characteristic_metrics(owner: str, repo: str, pr_number, developer_login: str, github_token: Optional[str] = None) -> Optional[Dict]:
    """
    Retrieves a developer's total commits and their percentage contribution to the repo.
    """
    
    # Setup headers with token
    headers = {}
    if github_token:
        headers["Authorization"] = f"token {github_token}"
    
    try:
        # 1. Developer's total commits in the repo (using the same pagination trick)
        developer_commit_url = f"https://api.github.com/repos/{owner}/{repo}/commits"
        params = {"per_page": 1, "author": developer_login}
        dev_response = safe_request("HEAD", developer_commit_url, headers=headers, params=params)
        
        developer_commits = 0
        if dev_response and dev_response.status_code == 200:
            dev_link_header = dev_response.headers.get('Link')
            if dev_link_header:
                dev_last_page_match = re.search(r'page=(\d+)>; rel="last"', dev_link_header)
                if dev_last_page_match:
                    developer_commits = int(dev_last_page_match.group(1))

        # 2. Total commits for the entire repository
        total_commits = get_commit_count(owner, repo, headers)
        
        # 3. Calculate the percentage contribution
        commit_percentage = (developer_commits / total_commits) * 100 if total_commits > 0 else 0.0
        
        # 4. Total PRs submitted by the developer 
        total_prs_submitted = get_developer_pr_count(owner, repo, developer_login, headers)
        
        # 5. Social and Account age metrics 
        social_metrics = get_developer_social_metrics(developer_login, headers)
        
        # 6. Total PRs submitted by the developer (Globally)
        total_prs_global = get_developer_global_pr_count(developer_login, headers)

        return {
            "Repo": f"{owner}/{repo}",
            "PR_ID": pr_number,
            "Developer_Login": developer_login,
            "Total_Repo_Commits": total_commits,
            "Developer_Commits": developer_commits,
            "Developer_Commit_Percentage": round(commit_percentage, 2),
            "Total_PRs_Submitted": total_prs_submitted, 
            "Total_PRs_Global": total_prs_global,        
            **social_metrics
        }

    except Exception as e:
        print(f"Error processing data for {developer_login} in {owner}/{repo}: {e}")
        return None
        
# ============================================================
# Main Helper Function: Developer Experience Metrics (With Caching)
# ============================================================
def fetch_metrics(pr_list: list, github_token: Optional[str], cached_filename: str) -> pd.DataFrame:
    """
    Fetches Developer Experience metrics with caching and incremental saving.
    The cache key is (owner, repo, pr_number, developer_login).
    """
    results = []
    
    # 1. Load Cache
    try:
        cached_df = pd.read_csv(cached_filename)
        # Create a unique key for caching/comparison
        cached_df['Key'] = cached_df.apply(lambda row: (row['owner'], row['repo'], row['pr_number'], row['developer_login']), axis=1)
        cached_keys = set(cached_df['Key'].tolist())
        print(f"Loaded {len(cached_keys)} existing metrics from {cached_filename}.")
        new_results = []
    except FileNotFoundError:
        cached_df = pd.DataFrame()
        cached_keys = set()
        print(f"No existing cache file found at {cached_filename}. Starting from scratch.")
        new_results = []

    # 2. Filter list to only process uncached entries
    # The input list is (owner, repo, pr_number, developer_login) tuples
    uncached_prs = [
        (owner, repo, pr_number, developer_login) 
        for owner, repo, pr_number, developer_login in pr_list 
        if (owner, repo, pr_number, developer_login) not in cached_keys
    ]
    
    print(f"Total PRs to process: {len(pr_list)}. Uncached PRs remaining: {len(uncached_prs)}.")

    # 3. Process uncached entries
    for i, (owner, repo, pr_number, developer_login) in enumerate(uncached_prs):
        # Progress printout
        if i % 50 == 0:
            print(f"Processing PR {i+1}/{len(uncached_prs)}: {owner}/{repo} #{pr_number} by {developer_login}")
        
        # Call the core metric function
        metrics = get_developer_characteristic_metrics(owner, repo, pr_number, developer_login, github_token)
        
        if metrics:
            # Add key columns to the results dictionary
            metrics['owner'] = owner
            metrics['repo'] = repo
            metrics['pr_number'] = pr_number
            metrics['developer_login'] = developer_login
            new_results.append(metrics)
        
        # Intermediate Save every 20 successful API calls (good practice for rate limit recovery)
        if (len(new_results) > 0) and (len(new_results) % 20 == 0):
            print(f"--- Saving intermediate progress: {len(new_results)} new entries...")
            
            # Combine new results with cached data and save
            temp_df = pd.DataFrame(new_results)
            updated_df = pd.concat([cached_df.drop(columns=['Key']), temp_df], ignore_index=True)
            subset_cols = ['owner', 'repo', 'pr_number', 'developer_login']
            updated_df = updated_df.drop_duplicates(subset=subset_cols, keep='last') 
            updated_df.to_csv(cached_filename, index=False)
            
            # Update the cached_df and cached_keys for the next iteration
            cached_df = updated_df
            cached_df['Key'] = cached_df.apply(lambda row: (row['owner'], row['repo'], row['pr_number'], row['developer_login']), axis=1)
            new_results = [] # Clear new results list

    # 4. Final Save
    if new_results:
        print(f"--- Final save: {len(new_results)} remaining new entries...")
        temp_df = pd.DataFrame(new_results)
        if not cached_df.empty:
            updated_df = pd.concat([cached_df.drop(columns=['Key']), temp_df], ignore_index=True)
        else:
            updated_df = temp_df
    else:
        updated_df = cached_df.drop(columns=['Key']) if not cached_df.empty else pd.DataFrame()

    subset_cols = ['owner', 'repo', 'pr_number', 'developer_login']
    updated_df = updated_df.drop_duplicates(subset=subset_cols, keep='last') 
    updated_df.to_csv(cached_filename, index=False)
    print(f"Metrics saved to {cached_filename}. Total entries: {len(updated_df)}.")
    
    # 5. Return the final full DataFrame
    return updated_df

# ============================================================
# MAIN PROGRAM
# ============================================================
print("\nStarting data retrieval... (may take a moment due to multiple API calls)")
pr_dev_df_accept = fetch_metrics(ACCEPTED_PULL_REQUEST, GITHUB_TOKEN, "pr_dev_metrics_accepted_cached.csv")
pr_dev_df_reject = fetch_metrics(REJECTED_PULL_REQUEST, GITHUB_TOKEN, "pr_dev_metrics_rejected_cached.csv")


Starting data retrieval... (may take a moment due to multiple API calls)
Error processing data for Copilot in microsoft/ApplicationInsights-Java: 422 Client Error: Unprocessable Entity for url: https://api.github.com/search/issues?q=repo%3Amicrosoft%2FApplicationInsights-Java+is%3Apr+author%3ACopilot&per_page=1
Error processing data for Copilot in microsoft/typespec: 422 Client Error: Unprocessable Entity for url: https://api.github.com/search/issues?q=repo%3Amicrosoft%2Ftypespec+is%3Apr+author%3ACopilot&per_page=1
Error processing data for Copilot in valkey-io/valkey-glide: 422 Client Error: Unprocessable Entity for url: https://api.github.com/search/issues?q=repo%3Avalkey-io%2Fvalkey-glide+is%3Apr+author%3ACopilot&per_page=1
Error processing data for Copilot in Azure/azure-sdk-for-java: 422 Client Error: Unprocessable Entity for url: https://api.github.com/search/issues?q=repo%3AAzure%2Fazure-sdk-for-java+is%3Apr+author%3ACopilot&per_page=1
Error processing data for Copilot in camun

# 4. Finalize

In [8]:
# ============================================================
# Helper: Finalize the dataframe, adding stars and forks
# ============================================================
def finalize_dataframe(metrics_df, output_filename):
    """
    Applies the merging, cleaning, renaming, and reordering steps 
    to a single metrics DataFrame.
    """
    
    # Define the rename mapping
    rename_map = {
        'PR_ID': 'PR_Number',
        'Developer_Login': 'User',
        'Total_Repo_Commits': 'Total_Commits_Repo', 
        'Developer_Commits': 'Dev_Commits_Repo', 
        'Developer_Commit_Percentage': 'Dev_Commits_Repo_Percentage', 
        'Total_PRs_Submitted': 'Dev_PR_Total', 
        'Account_Created_At': 'Creation_Date', 
        'Account_Active_Duration_Days': 'Days_Active', 
    }
    final_df = metrics_df.rename(columns=rename_map)

    # Apply the final column order
    final_df = final_df.fillna(0.0)

    # 4. Save the file (using CSV as per your original request)
    final_df.to_csv(output_filename, index=False)
    
    return final_df

# ============================================================
# MAIN PROGRAM - Separate Processing
# ============================================================

# --- Processing Accepted Repositories ---
print("\n--- Processing Accepted Repositories ---")
final_df_accept = finalize_dataframe(
    pr_dev_df_accept, 
    "pr_dev_metrics_accepted.csv" # Save to a separate file
)

print("\nAccepted Repository Metrics DataFrame Created:")
print(f"Total rows in Accepted DataFrame: {len(final_df_accept)}")

# --- Processing Rejected Repositories ---
print("\n--- Processing Rejected Repositories ---")
final_df_reject = finalize_dataframe(
    pr_dev_df_reject, 
    "pr_dev_metrics_rejected.csv" # Save to a separate file
)

print("\nRejected Repository Metrics DataFrame Created:")
print(f"Total rows in Rejected DataFrame: {len(final_df_reject)}")


--- Processing Accepted Repositories ---

Accepted Repository Metrics DataFrame Created:
Total rows in Accepted DataFrame: 36

--- Processing Rejected Repositories ---

Rejected Repository Metrics DataFrame Created:
Total rows in Rejected DataFrame: 45
